# Model training

In [119]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [53]:
df = pd.read_excel("data/dataset_cleaned.xlsx")
df

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,Nu_Siniestro_Relacionado,In_Fraude,De_Causa_Siniestro,...,wdir,wspd,pres,DESCRIPCION,month,month_sin,month_cos,day,day_sin,day_cos
0,1,7668,2015-05-29,1,29.0,1.0,Siniestro reportado vía cover.,,0,Accidente/Choque,...,63.0,9.7,1013.8,DISTRITO NACIONAL,5,5.000000e-01,-0.866025,29,-0.394356,0.918958
1,2,23643,2016-02-12,2,29.0,1.0,C/ MAYAGUEZ URB. VILLAS DUMIT,,0,Accidente/Choque,...,94.0,6.8,1016.4,SAN CRISTOBAL,2,8.660254e-01,0.500000,12,0.651372,-0.758758
2,3,29730,2016-02-09,3,1.0,1.0,Barahora,,0,Responsabilidad Civil Personas o Cosas,...,113.0,17.6,1018.3,BARAHONA,2,8.660254e-01,0.500000,9,0.968077,-0.250653
3,4,14301,2016-02-15,4,1.0,1.0,Arroyo Hondo II,,0,Robo o Hurto Accesorios/ Partes o Piezas,...,43.0,9.8,1015.8,DISTRITO NACIONAL,2,8.660254e-01,0.500000,15,0.101168,-0.994869
4,5,24144,2016-02-03,5,1.0,1.0,Avenida Gustavo Mejia Ricart,,0,Accidente/Choque,...,107.0,13.1,1015.4,DISTRITO NACIONAL,2,8.660254e-01,0.500000,3,0.571268,0.820763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,AUTOPISTA DUARTE,,0,Rotura de Vidrios y/o Parabrisas,...,43.0,20.0,1015.6,SANTO DOMINGO,4,8.660254e-01,-0.500000,4,0.724793,0.688967
18739,18703,181905,2023-06-03,11470,1.0,1.0,LOS MULTIS LOS ALCARRIZOS,,0,Rotura de Vidrios y/o Parabrisas,...,204.0,28.4,1009.6,SANTO DOMINGO,6,1.224647e-16,-1.000000,3,0.571268,0.820763
18740,18704,182109,2023-05-22,11471,1.0,1.0,AUTOPISTA DUARTE,,0,Rotura de Vidrios y/o Parabrisas,...,181.0,13.5,1011.7,SANTO DOMINGO,5,5.000000e-01,-0.866025,22,-0.968077,-0.250653
18741,18705,302838,2023-06-01,11472,1.0,1.0,CALLE CAMINO CHIQUITO,,0,Accidente/Choque,...,176.0,15.6,1013.7,DISTRITO NACIONAL,6,1.224647e-16,-1.000000,1,0.201299,0.979530


In [57]:
df.Cantidad_Parque_Vehicular.isnull().sum()

0

In [59]:
df.Cantidad_Parque_Vehicular.value_counts()

Cantidad_Parque_Vehicular
621291     1728
419442     1503
569102     1500
449918     1441
487157     1368
           ... 
2695457       1
21213         1
2573494       1
17018         1
3063704       1
Name: count, Length: 61, dtype: int64

In [68]:
# groupby_columns = ['Fe_Ocurrencia','De_Causa_Siniestro', 'Cd_Sexo', 'Relacion_Reclamante', 'In_Lista_Negra',
#                    'Marca', 'Modelo', 'Año', 'Clase',
#                    'Tipo_Dia', 'dia_feriado', 'latitude', #'Cantidad_Parque_Vehicular', 
#                    'longitude', 'PROVINCIA', 'Clima_Clear', 'Clima_Cloudy', 'Clima_Fair',
#                    'Clima_Fog', 'Clima_Rain', 'Clima_Thunderstorm', 'tavg', 'prcp', 'wdir',
#                    'wspd', 'pres', 'month_sin', 'month_cos', 'day_sin', 'day_cos']

# # Get the frequency of occurrences for each group
# frequency_data = df.groupby(groupby_columns).size().reset_index(name='frequency')
# frequency_data

In [129]:
groupby_columns = ['De_Causa_Siniestro','Tipo_Dia', 'dia_feriado','PROVINCIA', 'Clima_Clear', 'Clima_Cloudy', 'Clima_Fair',
                   'Clima_Fog', 'Clima_Rain', 'Clima_Thunderstorm', 'tavg', 'prcp', 'wdir',
                   'wspd', 'pres', 'month_sin', 'month_cos', 'day_sin', 'day_cos','ano','month','day']

# Get the frequency of occurrences for each group
frequency_data = df.groupby(groupby_columns).size().reset_index(name='frequency')
frequency_data

,De_Causa_Siniestro,Tipo_Dia,dia_feriado,PROVINCIA,Clima_Clear,Clima_Cloudy,Clima_Fair,Clima_Fog,Clima_Rain,Clima_Thunderstorm,...,wspd,pres,month_sin,month_cos,day_sin,day_cos,ano,month,day,frequency
0,Accidente/Choque,Weekday,0,BARAHONA,0,0,0,0,0,1,...,17.6,1018.3,0.866025,5.000000e-01,-0.897805,-0.440394,2018,2,21,1
1,Accidente/Choque,Weekday,0,BARAHONA,0,0,0,0,0,1,...,17.6,1018.3,-0.500000,8.660254e-01,0.101168,-0.994869,2018,11,15,1
2,Accidente/Choque,Weekday,0,BARAHONA,0,0,0,0,0,1,...,17.6,1018.3,-0.500000,8.660254e-01,-0.937752,0.347305,2015,11,25,1
3,Accidente/Choque,Weekday,0,BARAHONA,0,0,0,0,0,1,...,17.6,1018.3,-0.500000,8.660254e-01,0.201299,0.979530,2019,11,1,1
4,Accidente/Choque,Weekday,0,BARAHONA,0,0,0,0,0,1,...,17.6,1018.3,-0.500000,-8.660254e-01,-0.485302,-0.874347,2018,7,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11712,Rotura de Vidrios y/o Parabrisas,Weekend,1,SANTIAGO,0,0,0,0,1,0,...,10.9,1019.5,0.500000,8.660254e-01,0.201299,0.979530,2017,1,1,1
11713,Rotura de Vidrios y/o Parabrisas,Weekend,1,SANTO DOMINGO,0,0,0,0,1,0,...,6.3,1013.0,-1.000000,-1.836970e-16,-0.988468,0.151428,2022,9,24,1
11714,Rotura de Vidrios y/o Parabrisas,Weekend,1,SANTO DOMINGO,0,0,0,0,1,0,...,8.3,1016.1,0.500000,8.660254e-01,0.937752,0.347305,2018,1,6,1
11715,Rotura de Vidrios y/o Parabrisas,Weekend,1,SANTO DOMINGO,0,0,0,0,1,0,...,9.7,1017.8,0.500000,8.660254e-01,0.201299,0.979530,2017,1,1,1


In [130]:
frequency_data.frequency.value_counts()

frequency
1     8412
2     1948
3      756
4      333
5      148
6       76
7       26
8       11
9        4
11       1
10       1
30       1
Name: count, dtype: int64

### LabelEncoder for categorical variables and test train split

In [131]:
# Split the data into training and test sets
X = frequency_data.drop('frequency', axis=1)
y = frequency_data['frequency']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

# Define the numerical and categorical columns
num_columns = X.select_dtypes(include=['float64', 'int64']).columns
cat_columns = X.select_dtypes(include=['object']).columns

# Define the numerical and categorical transformers
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

# Create the column transformer
col_transformer = make_column_transformer(
    (num_transformer, num_columns),
    (cat_transformer, cat_columns),
    remainder='passthrough'
)

# Fit and transform the training data
X_train_processed = col_transformer.fit_transform(X_train)

# Transform the test data
X_test_processed = col_transformer.transform(X_test)

print(X_train_processed.shape)

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'int', 'str']

## Training

In [ ]:
# Lista de modelos de regresión
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('ElasticNet', ElasticNet()))
models.append(('SVR', SVR()))
models.append(('DecisionTree', DecisionTreeRegressor()))
models.append(('ExtraTrees', ExtraTreesRegressor()))
models.append(('AdaBoost', AdaBoostRegressor()))
models.append(('Bagging', BaggingRegressor()))
models.append(('BayesianRidge', BayesianRidge()))
models.append(('GradientBoosting', GradientBoostingRegressor()))
models.append(('RandomForest', RandomForestRegressor()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('XGBoost', XGBRegressor()))
# models.append(('LightGBM', LGBMRegressor()))
# models.append(('CatBoost', CatBoostRegressor()))

In [ ]:
# Guardamos los resultados del modelo
resultados = []
nombres = []
scoring = 'neg_root_mean_squared_error'
# como métrica de puntuación en Scikit-learn, estás expresando tu deseo de maximizar el valor negativo del MSE, lo que equivale a minimizar el MSE real.

for nombre, model in models:
    kfold = KFold(n_splits=10, random_state=None)
    cv_resultados = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    resultados.append(cv_resultados)
    nombres.append(nombre)
    msg = ('{}: {} ({})'.format(nombre, cv_resultados.mean(), cv_resultados.std()))
    print(msg)

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Aneur\Miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Aneur\Miniconda3\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Aneur\Miniconda3\lib\site-packages\sklearn\linear_model\_base.py", line 678, in fit
    X, y = self._validate_data(
  File "c:\Users\Aneur\Miniconda3\lib\site-packages\sklearn\base.py", line 621, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\Users\Aneur\Miniconda3\lib\site-packages\sklearn\

In [ ]:
print(X_train.dtypes)


Fe_Ocurrencia         datetime64[ns]
De_Causa_Siniestro           float64
Tipo_Dia                     float64
dia_feriado                  float64
PROVINCIA                    float64
Clima_Clear                  float64
Clima_Cloudy                 float64
Clima_Fair                   float64
Clima_Fog                    float64
Clima_Rain                   float64
Clima_Thunderstorm           float64
tavg                         float64
prcp                         float64
wdir                         float64
wspd                         float64
pres                         float64
month_sin                    float64
month_cos                    float64
day_sin                      float64
day_cos                      float64
dtype: object


In [ ]:
#  Graficamos para tener mejor ilustracion
fig = plt.figure()
fig.suptitle('Comparación de algoritmos')
ax = fig.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(nombres)
plt.show()